# 分散表現を特徴量としたクラスタリング

- word2vecのベクトルの平均値をテキストのベクトルとしてクラスタリング

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# Spacyの日本語モデルをロード
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み
feeds = pd.read_csv('data/output_jp.csv')

# title から　'\u300012/4 10:35更新'　の部分を削除
feeds['title'] = feeds['title'].map(lambda x: x[:x.rfind('\u3000')] if x[-2:] == '更新' else x)

# text と summary を結合
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/output_jp.csv'

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化

- word2vecのベクトルの平均値をテキストのベクトルとする

In [ ]:
# テキストのトークンのword2vecベクトルの平均値を計算
def text2vec(text):
    tokens = preprocess(text)
    total = nlp.vocab[' '].vector
    n = len(tokens)
    for t in tokens:
        vec = nlp.vocab[t].vector
        total = total + vec
    # ndarrayのベクトルをlistにして返す
    return (total / n).tolist()

# mapの結果はSeriesになるため、ndarrayからlistに変換することでlistのlistができ、
# それをDataFrameにする
vector = pd.DataFrame(df['text'].map(lambda x: text2vec(x)).values.tolist())
# 確認
vector

### 非階層的クラスタリング

In [ ]:
# クラスタ数
N_clusters = 10

# KMeans でクラスタリング
# - normalize() により単位ベクトル化
clusters = KMeans(n_clusters=N_clusters).fit_predict(normalize(vector))

# 結果を DataFrame にまとめる
df_cluster = pd.DataFrame(clusters, columns=['cluster'])

### クラスタごとの棒グラフ（頻度）

In [ ]:
# 単語を頻度を計算するためにCountVectorizerを利用
vectorizer = CountVectorizer(tokenizer=preprocess)
bow_vector = vectorizer.fit_transform(df.text)

# 単語の DataFrame
df_words = pd.DataFrame(vectorizer.get_feature_names_out())

# テキストをベクトル化した結果の DataFrame
df_vector = pd.DataFrame.sparse.from_spmatrix(bow_vector)

In [ ]:
# 頻度の棒グラフ
for i in range(0, N_clusters):
    print('Cluster', i)
    _cdf = df_cluster[df_cluster['cluster'] == i]
    df_counts = pd.concat([df_words, df_vector.iloc[_cdf.index].sum()], axis=1)
    df_counts.columns=['word', 'counts']
    df_bar = df_counts.sort_values('counts', ascending=False).head(10)
    sns.barplot(x=df_bar.counts, y=df_bar.word, orient='h')
    plt.show()